In [8]:
import pandas as pd
import json
from itertools import islice

In [17]:
def create_chunks(path):
    total = 0
    with open(path) as f:
        for i in range(0, 20):
            lines = list(islice(f, 100_000 * i, 100_000 * (i + 1)))
            data = [json.loads(line[:-2]) for line in lines[1:]]
            df = pd.DataFrame(data)
            df = (
                df[[
                    "id", 
                    "year", 
                    "authors", 
                    "venue", 
                    "keywords",
                    "title", 
                    "abstract",
                    "n_citation",
                    "references",
                    "lang"
                ]]
                .loc[df["lang"] == "en"]
                .dropna()
            )
            df["venue"] = df["venue"].apply(lambda x: x["raw"])
            df["abstract"] = df["abstract"].astype(str)
            df = df[df["abstract"].apply(lambda x: len(x) > 10)]
            df = df[df["references"].apply(lambda x: len(x) > 1)]
            df = df[df["keywords"].apply(lambda x: len(x) > 1)]
            df = df[df["venue"].apply(lambda x: len(x) > 1)]
            df["text"] = (
                df["title"] + "\n" 
                # + df["authors.name"] + "\n" 
                # + df["author.org"] + "\n" 
                + df["venue"] + "\n" 
                # + df["fos.name"] + "\n" 
                + df["keywords"].apply(lambda x: " ".join(x)) + "\n"
                + df["abstract"]
            )
            df = df.sort_values(by="id").reset_index(drop=True)
            df.to_feather(f"../data/data-{i}.feather")
            total += len(df)
            print(f"Chunk {i} done. Total: {total}")
            if total > 100_000:
                break

create_chunks("../data/dblp_v14.json")

Chunk 0 done. Total: 45444
Chunk 1 done. Total: 97115
Chunk 2 done. Total: 160255


In [20]:
a = pd.read_feather("../data/data-0.feather")
b = pd.read_feather("../data/data-1.feather")
c = pd.read_feather("../data/data-2.feather")
d = pd.concat([a, b, c])
d

,id,year,authors,venue,keywords,title,abstract,n_citation,references,lang,text
0,53e99784b7602d9701f3e15d,2005,"[{'id': '53f43b03dabfaedce555bf2a', 'name': 'M...",ISCAS (3),"[sequential circuits, statistical distribution...",Timing yield estimation using statistical stat...,As process variations become a significant pro...,32,"[53e99af2b7602d970237f6ee, 53e99ebdb7602d97027...",en,Timing yield estimation using statistical stat...
1,53e99784b7602d9701f3f411,2002,"[{'id': '548a2e3ddabfae9b40134fbc', 'name': 'H...",COMPSAC,"[Internet, hypermedia markup languages, inform...",Using XML to Integrate Existing Software Syste...,The eXtensible Markup Language 驴 XML 驴 is not ...,28,"[53e997f5b7602d9701ff904f, 53e9a326b7602d9702c...",en,Using XML to Integrate Existing Software Syste...
2,53e99784b7602d9701f3f5fe,2011,"[{'id': '53f46a22dabfaee0d9c3d5e5', 'name': 'S...",Frontiers of Computer Science in China,"[resource allocation, cpu utilization, quality...",Research on resource allocation for multi-tier...,Resource allocation for multi-tier web applica...,2,"[53e998cdb7602d97021044db, 53e9992bb7602d97021...",en,Research on resource allocation for multi-tier...
3,53e99784b7602d9701f3f636,2002,"[{'id': '53f58b15dabfaece00f8046d', 'name': 'P...",Commun. ACM,"[linear scale, false dichotomy]",Flatlined,Our propensity to create linear scales between...,7,"[53e99b0ab7602d970239cabe, 53e99f7eb7602d97028...",en,Flatlined\nCommun. ACM\nlinear scale false dic...
4,53e99784b7602d9701f3f669,2006,"[{'id': '5435290fdabfaebba58aa917', 'name': 'J...","J. Comb. Theory, Ser. B","[homomorphism order, Minor, Kk-free graph, gen...",Folding,We define folding of a directed graph as a col...,16,"[53e9a381b7602d9702c8cf9a, 53e9a6adb7602d9702f...",en,"Folding\nJ. Comb. Theory, Ser. B\nhomomorphism..."
...,...,...,...,...,...,...,...,...,...,...,...
63135,568687960cf27f35d584d526,2016,"[{'id': '53f42f34dabfaeb1a7ba2a13', 'name': 'K...",Neural Networks,"[2-norm, Disjoint, LRR, Subspace segmentation]",Subspace segmentation by dense block and spars...,Subspace segmentation is a fundamental topic i...,25,"[53e99b0ab7602d970239def1, 53e99df7b7602d97026...",en,Subspace segmentation by dense block and spars...
63136,56d91461dabfae2eee501940,2009,"[{'id': '53f43207dabfaee02ac9bad5', 'name': 'G...",European Journal of Information Systems,"[information security, strategic information s...",Another road to IT turnover: the entrepreneuri...,"This paper addresses an untapped, though impor...",55,"[53e99905b7602d97021418d0, 53e99b4ab7602d97023...",en,Another road to IT turnover: the entrepreneuri...
63137,599c7980601a182cd264555f,2017,"[{'id': '53f43771dabfaee2a1cefdaa', 'name': 'T...",IEEE Transactions on Information Theory,"[Additive code, subfield subcode, subspace sub...",Generalized subspace subcodes with application...,Most codes with an algebraic decoding algorith...,9,"[53e9a4ddb7602d9702e002d0, 53e99d13b7602d97025...",en,Generalized subspace subcodes with application...
63138,599c7c68601a182cd27aaf1c,2017,"[{'id': '', 'name': 'Amit Rama Akula', 'org': ...",Conf. Computing Frontiers,"[Integrated Cloud Management, Advanced Manufa...",Advanced Manufacturing Collaboration in a Clou...,Advanced Manufacturing Apps that perform compl...,2,"[53e9a8b7b7602d9703204d73, 53e9b153b7602d9703b...",en,Advanced Manufacturing Collaboration in a Clou...


In [77]:
with open('../data/dblp_v14.json') as f:
    lines = f.readlines(int(1e8))
    data = [json.loads(line[:-2]) for line in lines[1:]]

In [90]:
df = pd.DataFrame(data)
df

,id,title,doi,issue,keywords,lang,venue,year,n_citation,page_start,...,isbn,url,abstract,authors,doc_type,references,fos,indexed_abstract,v12_id,v12_authors
0,53e99784b7602d9701f3ffdd,Flow.,10.1145/1280120.1280225,10,[],en,{'raw': 'SIGGRAPH Art Gallery'},2006,15,,...,,[],,"[{'id': '53f43776dabfaee0d9b6e75b', 'name': 'M...",Conference,NaN,NaN,NaN,NaN,NaN
1,53e99785b7602d9701f41492,Dancing,10.1145/281388.281845,19,[],en,{'raw': 'ACM SIGGRAPH 98 Electronic art and an...,1998,0,116,...,1-58113-045-7,"[http://dx.doi.org/10.1145/281388.281845, http...",,"[{'id': '53f42e86dabfaee43ebd375a', 'name': 'A...",Conference,NaN,NaN,NaN,NaN,NaN
2,53e99785b7602d9701f414a7,Documentation,10.1007/3-540-07168-7_82,,[],en,"{'raw': 'Software Engineering, An Advanced Cou...",1972,3,385,...,3-540-07168-7,[],,"[{'id': '53f4671bdabfaeee22a56233', 'name': 'G...",Conference,NaN,NaN,NaN,NaN,NaN
3,53e99785b7602d9701f414a8,Datennetze,,,[],en,{'raw': 'Informatik: Grundlagen - Amwendungen ...,1993,0,32,...,3-406-40338-7,[],,"[{'id': '53f4d423dabfaeedd9781e23', 'name': 'O...",Conference,NaN,NaN,NaN,NaN,NaN
4,53e99785b7602d9701f414cb,Drift,10.1145/1186015.1186062,06,[],en,{'raw': 'SIGGRAPH '04 ACM SIGGRAPH 2004 Comput...,2004,0,209,...,1-58113-896-2,[],,"[{'id': '53f433e6dabfaeee229920b6', 'name': 'J...",Conference,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41219,53e99813b7602d970202cbfa,Distinguished experts panel,10.1109/NOMS.2008.4575097,,"[intelligent sensors, ubiquitous computing, en...",en,{'raw': 'NOMS'},2008,0,,...,,[http://dx.doi.org/10.1109/NOMS.2008.4575097],,"[{'id': '', 'name': 'George Pavlou', 'org': ''...",Conference,[],"[{'name': 'Information system', 'w': 0.5064}, ...","{'IndexLength': 162, 'InvertedIndex': {'increa...",1.859909e+09,"[{'name': 'G. Pavlou', 'org': 'University Coll..."
41220,53e99813b7602d970202d559,Digital Khalimsky Manifolds,10.1007/s10851-008-0114-1,3,"[Khalimsky topology, Digital manifold, Join op...",en,{'raw': 'Journal of Mathematical Imaging and V...,2009,10,267,...,,"[http://dx.doi.org/10.1007/s10851-008-0114-1, ...",We consider different possibilities to define ...,"[{'id': '53f42b1adabfaedce54a061e', 'name': 'E...",Journal,"[53e9bb1cb7602d9704754471, 53e99d28b7602d97025...","[{'name': 'Adjacency list', 'w': 0.52308}, {'n...","{'IndexLength': 55, 'InvertedIndex': {'possibi...",1.997855e+09,"[{'name': 'Erik Melin', 'org': 'Department of ..."
41221,53e99813b7602d970202d558,Das KARAMBA Methodenbanksystem,10.1007/978-3-642-67444-0_29,,[das karamba methodenbanksystem],en,{'raw': 'GI Jahrestagung'},1979,3,322,...,3-540-09664-7,[http://dx.doi.org/10.1007/978-3-642-67444-0_29],,"[{'id': '53f4d278dabfaef0eff81219', 'name': 'R...",Conference,"[53e9b6c4b7602d970424b1ad, 53e99b26b7602d97023...",[],"{'IndexLength': 79, 'InvertedIndex': {'die': [...",1.556374e+09,"[{'name': 'Rainer Hüber', 'org': 'Universität ..."
41222,53e99813b7602d970202d55a,Discriminative k-metrics,10.1145/1553374.1553503,,"[affine set, actual classification task, basic...",en,{'raw': 'ICML'},2009,30,127,...,,"[http://dx.doi.org/10.1145/1553374.1553503, ht...",The k q-flats algorithm is a generalization of...,"[{'id': '53f46264dabfaefedbb793a6', 'name': 'A...",Conference,"[53e9981db7602d970203bc0e, 53e99924b7602d97021...","[{'name': 'Affine transformation', 'w': 0.5998...","{'IndexLength': 111, 'InvertedIndex': {'standa...",2.118744e+09,"[{'name': 'Arthur Szlam', 'org': 'UCLA, Los An..."


In [93]:
df.to_json('../data/dblp.json', orient='records', lines=True)

In [88]:
df = (
    df[[
        "id", 
        "year", 
        "authors", 
        "venue", 
        "keywords",
        "title", 
        "abstract",
        "n_citation",
        "references",
        "lang"
    ]]
    .loc[df["lang"] == "en"]
    .dropna()
    # .sort_values(by="id")
    # .reset_index(drop=True)
)
df

,id,year,authors,venue,keywords,title,abstract,n_citation,references,lang
16,53e9978ab7602d9701f47342,1993,"[{'id': '53f4757ddabfaee4dc88a39f', 'name': 'F...",{'raw': 'Journal of Philosophical Logic'},[],Causation,As cause we often specify an event the occurre...,19,[],en
22,53e9978ab7602d9701f4bc56,2007,"[{'id': '53f4668fdabfaee43ecf7323', 'name': 'C...",{'raw': 'European Conference on Object-Oriente...,"[Nick Levine, relevant organisational aspect, ...",Lisp,This report covers the activities of the 3rd E...,2,[53e99a04b7602d970225216a],en
27,53e9978db7602d9701f4e881,1994,"[{'id': '53f5a146dabfaeffbaf8048a', 'name': 'M...",{'raw': 'ACM SIGPLAN Notices'},[],J+=J,,0,"[53e9a81fb7602d97031681bc, 53e9b159b7602d9703b...",en
28,53e9978db7602d9701f4e899,2004,"[{'id': '560179fd45cedb3395e5f24e', 'name': 'N...",{'raw': 'Nature'},"[Nature, science, science news, biology, p...",Japan.,Whether military intervention for humanitarian...,0,[],en
32,53e9978db7602d9701f50771,2016,"[{'id': '548a36a2dabfae9b4013506a', 'name': 'R...",{'raw': 'interactions'},"[transfer operator, generalized correspondence...",interactions,"Given a C ∗ -algebra B , a closed * -subalgebr...",6,[],en
...,...,...,...,...,...,...,...,...,...,...
41219,53e99813b7602d970202cbfa,2008,"[{'id': '', 'name': 'George Pavlou', 'org': ''...",{'raw': 'NOMS'},"[intelligent sensors, ubiquitous computing, en...",Distinguished experts panel,,0,[],en
41220,53e99813b7602d970202d559,2009,"[{'id': '53f42b1adabfaedce54a061e', 'name': 'E...",{'raw': 'Journal of Mathematical Imaging and V...,"[Khalimsky topology, Digital manifold, Join op...",Digital Khalimsky Manifolds,We consider different possibilities to define ...,10,"[53e9bb1cb7602d9704754471, 53e99d28b7602d97025...",en
41221,53e99813b7602d970202d558,1979,"[{'id': '53f4d278dabfaef0eff81219', 'name': 'R...",{'raw': 'GI Jahrestagung'},[das karamba methodenbanksystem],Das KARAMBA Methodenbanksystem,,3,"[53e9b6c4b7602d970424b1ad, 53e99b26b7602d97023...",en
41222,53e99813b7602d970202d55a,2009,"[{'id': '53f46264dabfaefedbb793a6', 'name': 'A...",{'raw': 'ICML'},"[affine set, actual classification task, basic...",Discriminative k-metrics,The k q-flats algorithm is a generalization of...,30,"[53e9981db7602d970203bc0e, 53e99924b7602d97021...",en


In [89]:
df["venue"] = df["venue"].apply(lambda x: x["raw"])
df["abstract"] = df["abstract"].astype(str)
df = df[df["abstract"].apply(lambda x: len(x) > 10)]
df = df[df["references"].apply(lambda x: len(x) > 1)]
df = df[df["keywords"].apply(lambda x: len(x) > 1)]
df = df[df["venue"].apply(lambda x: len(x) > 1)]
df

,id,year,authors,venue,keywords,title,abstract,n_citation,references,lang
56,53e9979bb7602d9701f66d44,2001,"[{'id': '54092851dabfae92b425e686', 'name': 'D...",Handbook of logic in artificial intelligence a...,"[newspaper story warning, reasonable person, c...",Defeasible logic,We often reach conclusions partially on the ba...,370,"[53e9979bb7602d9701f66d44, 53e9981db7602d97020...",en
62,53e9979eb7602d9701f6ad81,2009,"[{'id': '53f43023dabfaedf435394fb', 'name': 'M...",Computing with instinct,"[Exploitational interaction, Exploitational In...",Exploitational interaction,Exploitational Interaction is an accessibility...,1,"[558ae60584ae84d265c04af2, 53e997f1b7602d9701f...",en
65,53e9979eb7602d9701f6ee80,1997,"[{'id': '53f479e8dabfae8a6845c98b', 'name': 'A...",Journal of Cryptology,"[new scheme, rsa scheme, modular multiplicatio...",Batch RSA,We present a variant of the RSA algorithm call...,117,"[557d1a22f667eeed56197990, 557e82ea6fee0fe990c...",en
70,53e997a2b7602d9701f767a7,2001,"[{'id': '54871e22dabfae9b401341fe', 'name': 'K...","Combinatorial optimization - Eureka, you shrink!","[G. Plummer, bipartite graph, famous Hadwiger ...",Connected matchings,A connected matching M in a graph G is a match...,3,"[53e9a96db7602d97032c6e1b, 53e9aa5cb7602d97033...",en
71,53e997a2b7602d9701f77696,1996,"[{'id': '543149d1dabfae8f291413ae', 'name': 'M...",Computational Learning Theory,"[PAC setting, CLASSIC sentence, CLASSIC learni...",CLASSIC learning,"Description logics, also called terminological...",103,"[53e997fcb7602d9702005ad4, 53e99842b7602d97020...",en
...,...,...,...,...,...,...,...,...,...,...
41211,53e99813b7602d970202d1ab,2011,"[{'id': '53f4413cdabfaee43ec6e4e8', 'name': 'S...",ACL,"[foreign text, foreign language, decipherment ...",Deciphering Foreign Language,"In this work, we tackle the task of machine tr...",143,"[5c782e764895d9cbc6829741, 53e998a3b7602d97020...",en
41214,53e99813b7602d970202d4ca,1995,"[{'id': '53f3b1e7dabfae4b34b2139e', 'name': 'S...",ICCV,"[kinetic occlusion, spatiotemporal imagery, sp...",Detecting kinetic occlusion,Visual motion boundaries provide a powerful cu...,59,"[557f98f7f6678c77ea22ae9d, 558271b00cf2e37586b...",en
41215,53e99813b7602d970202d4cb,2009,"[{'id': '5405866edabfae91d3fed3d4', 'name': 'S...",FOMI,"[ontological technique, disentangling knowledg...",Disentangling Knowledge Objects,We discuss a framework for knowledge managemen...,5,"[53e9ac55b7602d9703628de7, 53e9be21b7602d9704a...",en
41220,53e99813b7602d970202d559,2009,"[{'id': '53f42b1adabfaedce54a061e', 'name': 'E...",Journal of Mathematical Imaging and Vision,"[Khalimsky topology, Digital manifold, Join op...",Digital Khalimsky Manifolds,We consider different possibilities to define ...,10,"[53e9bb1cb7602d9704754471, 53e99d28b7602d97025...",en


In [81]:
df["text"] = (
    df["title"] + "\n" 
    # + df["authors.name"] + "\n" 
    # + df["author.org"] + "\n" 
    + df["venue"] + "\n" 
    # + df["fos.name"] + "\n" 
    + df["keywords"].apply(lambda x: " ".join(x)) + "\n"
    + df["abstract"]
)

print(df["text"].iloc[0])

In [84]:
df

,id,year,authors,venue,keywords,title,abstract,n_citation,references,lang,text
56,53e9979bb7602d9701f66d44,2001,"[{'id': '54092851dabfae92b425e686', 'name': 'D...",Handbook of logic in artificial intelligence a...,"[newspaper story warning, reasonable person, c...",Defeasible logic,We often reach conclusions partially on the ba...,370,"[53e9979bb7602d9701f66d44, 53e9981db7602d97020...",en,Defeasible logic\nHandbook of logic in artific...
62,53e9979eb7602d9701f6ad81,2009,"[{'id': '53f43023dabfaedf435394fb', 'name': 'M...",Computing with instinct,"[Exploitational interaction, Exploitational In...",Exploitational interaction,Exploitational Interaction is an accessibility...,1,"[558ae60584ae84d265c04af2, 53e997f1b7602d9701f...",en,Exploitational interaction\nComputing with ins...
65,53e9979eb7602d9701f6ee80,1997,"[{'id': '53f479e8dabfae8a6845c98b', 'name': 'A...",Journal of Cryptology,"[new scheme, rsa scheme, modular multiplicatio...",Batch RSA,We present a variant of the RSA algorithm call...,117,"[557d1a22f667eeed56197990, 557e82ea6fee0fe990c...",en,Batch RSA\nJournal of Cryptology\nnew scheme r...
70,53e997a2b7602d9701f767a7,2001,"[{'id': '54871e22dabfae9b401341fe', 'name': 'K...","Combinatorial optimization - Eureka, you shrink!","[G. Plummer, bipartite graph, famous Hadwiger ...",Connected matchings,A connected matching M in a graph G is a match...,3,"[53e9a96db7602d97032c6e1b, 53e9aa5cb7602d97033...",en,Connected matchings\nCombinatorial optimizatio...
71,53e997a2b7602d9701f77696,1996,"[{'id': '543149d1dabfae8f291413ae', 'name': 'M...",Computational Learning Theory,"[PAC setting, CLASSIC sentence, CLASSIC learni...",CLASSIC learning,"Description logics, also called terminological...",103,"[53e997fcb7602d9702005ad4, 53e99842b7602d97020...",en,CLASSIC learning\nComputational Learning Theor...
...,...,...,...,...,...,...,...,...,...,...,...
41214,53e99813b7602d970202d4ca,1995,"[{'id': '53f3b1e7dabfae4b34b2139e', 'name': 'S...",ICCV,"[kinetic occlusion, spatiotemporal imagery, sp...",Detecting kinetic occlusion,Visual motion boundaries provide a powerful cu...,59,"[557f98f7f6678c77ea22ae9d, 558271b00cf2e37586b...",en,Detecting kinetic occlusion\nICCV\nkinetic occ...
41215,53e99813b7602d970202d4cb,2009,"[{'id': '5405866edabfae91d3fed3d4', 'name': 'S...",FOMI,"[ontological technique, disentangling knowledg...",Disentangling Knowledge Objects,We discuss a framework for knowledge managemen...,5,"[53e9ac55b7602d9703628de7, 53e9be21b7602d9704a...",en,Disentangling Knowledge Objects\nFOMI\nontolog...
41217,53e99813b7602d970202d2ef,2009,"[{'id': '53f44530dabfaedd74de9493', 'name': 'S...",,"[data web, dbpedia project extracts informatio...",DBpedia Live Extraction,The DBpedia project extracts information from ...,92,"[53e997c1b7602d9701fadd55, 53e998fdb7602d97021...",en,DBpedia Live Extraction\n\ndata web dbpedia pr...
41220,53e99813b7602d970202d559,2009,"[{'id': '53f42b1adabfaedce54a061e', 'name': 'E...",Journal of Mathematical Imaging and Vision,"[Khalimsky topology, Digital manifold, Join op...",Digital Khalimsky Manifolds,We consider different possibilities to define ...,10,"[53e9bb1cb7602d9704754471, 53e99d28b7602d97025...",en,Digital Khalimsky Manifolds\nJournal of Mathem...


In [86]:
df = df.sort_values(by="id").reset_index(drop=True)
df.to_feather("../data/data.feather")